# Getting started

To get started please make sure you have followed the [installation instructions][installation].

In this tutorial you will learn how to:

1. Create a simple electrical network with one source and one load
2. Run a load flow simulation
3. Get the results
4. Update the elements of the network
5. Save the network and the results to the disk for later analysis
6. Load the saved network and the results from the disk

Let's start by importing everything we will need.

[installation]: https://roseautechnologies.github.io/Roseau_Load_Flow/Installation

In [ ]:
import getpass

import numpy as np

from roseau.load_flow import *

## Creating a network

An electrical network can be built by assembling basic elements provided in
[roseau.load_flow.models][api-models]. The following is a summary on the available elements:

* Buses:
  * `Bus`: An electrical bus.

* Branches:
  * `Line`: A line connects two buses. The parameters of the line are defined by a `LineParameters` object.
  * `LineParameters`: This object defines the parameters of a line  (model, impedance, etc.)
  * `Switch`: A basic switch element.
  * `Transformer`: A generic transformer. The parameters of the transformer are defined by a `TransformerParameters` object.
  * `TransformerParameters`: This object defines the parameters of a transformer (model, windings, etc.)
* Loads:
  The ZIP load model is available via the following classes:
  * `ImpedanceLoad`: A constant impedance (Z) load: $S = |V|^2 \times \overline{Z}$, $|S|$ is proportional to $|V|^2$.
  * `CurrentLoad` A constant impedance (I) load: $S = V \times \overline{I}$, $|S|$ is proportional to $|V|^1$.
  * `PowerLoad`: A constant power (P) load: $S = \mathrm{constant}$, $|S|$ is proportional to $|V|^0$.

  A power load can be made flexible (controllable) by using the following class:
  * `FlexibleParameter`: This object defines the parameters of the flexible load's control (Maximum power, projection, type, etc.)
* Sources:
  * `VoltageSource`: A voltage source has a constant voltage and an infinite power source.
* Others:
  * `Ground`: A ground acts as a perfect conductor. If two elements are connected to the ground, the potentials at the connection points are always equal.
  * `PotentialRef`: A potential reference sets the reference of potentials in the network. It can be connected to buses or grounds.

Let's use some of these elements to build a network with a voltage source, a simple line and a constant power load.
This network is a low voltage network (three-phase +  neutral wire).

[api-models]: https://roseautechnologies.github.io/Roseau_Load_Flow/autoapi/roseau/load_flow/models


In [ ]:
# Create two buses
source_bus = Bus(id="sb", phases="abcn")
load_bus = Bus(id="lb", phases="abcn")

# Define the reference of potentials to be the neutral of the source bus
ground = Ground(id="gnd")
pref = PotentialRef(id="pref", element=ground)  # Set the potential of the ground element to 0
ground.connect(source_bus, phase="n")

# Create a LV source at the first bus
un = 400 / np.sqrt(3)  # Volts (phase-to-neutral because the source is connected to the neutral)
source_voltages = [un, un * np.exp(-2j * np.pi / 3), un * np.exp(2j * np.pi / 3)]
vs = VoltageSource(id="vs", bus=source_bus, phases="abcn", voltages=source_voltages)

# Add a load at the second bus
load = PowerLoad(id="load", bus=load_bus, phases="abcn", powers=[10e3 + 0j, 10e3, 10e3])  # VA

# Add a LV line between the source bus and the load bus
lp = LineParameters("lp", z_line=0.1 * np.eye(4, dtype=complex))  # R = X = 0.1 Ohm/km
line = Line(id="line", bus1=source_bus, bus2=load_bus, phases="abcn", parameters=lp, length=2.0)

At this point, all the basic elements of the network have been defined and connected. Now,
everything can be encapsulated in an `ElectricalNetwork` object, but first, some important
notes on the `Ground` and `PotentialRef` elements:

<div class="alert alert-info">

**Important:**

The `Ground` element does not have a fixed potential as one would expect from a real ground
connection. The potential reference (0 Volts) is defined by the `PotentialRef` element that
itself can be connected to any bus or ground in the network. This is to give more flexibility
for the user to define the potential reference of their network.

The `PotentialRef` defines the potential reference for this network. This is a mandatory
reference for the load flow resolution to be well defined. A network MUST have one and only
one potential reference per a galvanically isolated section.
</div>

<div class="alert alert-info">

**Note:**

The `Ground` element is not required in this simple network as it is connected to a single
element. No current will flow through the ground and no two points in the network will be forced
to have the same potential. In this scenario you are allowed to define the potential reference
directly on the bus element: `pref = PotentialRef(id="pref", element=source_bus, phase="n")` and
not bother with defining the ground element at all.
</div>

The `ElectricalNetwork` object can now be created using the `from_element` constructor. The
source bus `source_bus` is given to this constructor. All the elements connected to this bus are
automatically included into the network.

In [ ]:
# Create the electrical network
en = ElectricalNetwork.from_element(source_bus)

## Solving a load flow

An authentication is required. Please contact us at contact@roseautechnologies.com to get the necessary credentials.

<div class="alert alert-warning">

**Warning:**

`input` should never be used to read passwords and secret credentials as the terminal might save the password in
plain text in its history. Use `getpass.getpass` that provides the necessary protections instead.
</div>


In [ ]:
# Authentication
auth = (input("Please enter your username:"), getpass.getpass("Please enter your password:"))

Then, the load flow can be solved by requesting our server (requires Internet access).

<div class="alert alert-info">

**Note:**

The server takes some time to warm up the first time it is requested. Subsequent requests will
execute faster.
</div>


In [ ]:
# Solve the load flow
en.solve_load_flow(auth=auth)

It returns the number of iterations performed by the *Newton-Raphson* algorithm. Here, 2.

## Getting the results

The results are now available for every element of the network. Results can be accessed through
special properties prefixed with `res_` on each element object. For instance, the potentials
of the `load_bus` can be accessed using the property `load_bus.res_potentials`. It contains 4
values which are the potentials of its phases `a`, `b`, `c` and `n` (neutral). This potentials
returned are complex numbers. Calling `abs(load_bus.res_potentials)` gives you the magnitude of
the load's potentials (in Volts) and `np.angle(load_bus.res_potentials)` gives their angle in
radians.


<div class="alert alert-info">

**Note:**

Roseau Load Flow uses the [Pint](https://pint.readthedocs.io/en/stable/) `Quantity` objects to
handle data in unit-agnostic way. All input data (load powers, source voltages, etc.) is
expected to be either given in SI units or using the pint Quantity interface for non-SI units
(example below). The `length` paramater of the `Line` class that is an exception where the
default unit is Kilometers.

Example, create a load with powers expressed in kVA:
```python
from roseau.load_flow import Q_

load = PowerLoad(id="load", bus=load_bus, phases="abcn", powers=Q_([10, 10, 10], "kVA"))
```
</div>

The results returned by the `res_` properties are also `Quantity` objects.

### Available results

The available results depend on the type of element. The following table summarizes the available
results for each element type:

| Element type | Available results |
|--------------|-------------------|
| `Bus` | `res_potentials`, `res_voltages` |
| `Line` | `res_currents`, `res_powers`, `res_potentials`, `res_voltages`, `res_series_power_losses`, `res_shunt_power_losses`, `res_power_losses` |
| `Transformer`, `Switch` | `res_currents`, `res_powers`, `res_potentials`, `res_voltages` |
| `ImpedanceLoad`, `CurrentLoad`, `PowerLoad` | `res_currents`, `res_powers`, `res_potentials`, `res_voltages` |
| `VoltageSource` | `res_currents`, `res_powers`, `res_potentials`, `res_voltages` |
| `Ground` | `res_potential` |
| `PotentialRef` | `res_current` *(Always zero for a successful load flow)* |


### Getting results per object

In order to get the potentials or voltages of a bus, use the `res_potentials` or `res_voltages`
properties of buses as follows:

In [ ]:
load_bus.res_potentials

The currents of the line are available using the `res_currents` property of the `line` object.
It contains two arrays:

* the first is the current flowing from the first bus of the line to the second bus of the line.
  It contains 4 values: one per phase and the neutral current.
* the second is the current flowing from the second bus of the line to the first bus of the line.

Here, the sum of these currents is 0 as we have chosen a simple line model, i.e, a line with only
series impedance elements without shunt. If a shunt was present, the sum would have been non-zero.

In [ ]:
line.res_currents

For a flexible load (a `PowerLoad` with `flexible_params`), the final power values can be
retrieved using the property `res_flexible_powers`.

### Dataframe network results

The results can also be retrieved for the entire network using `res_` properties of the
`ElectricalNetwork` instance as [pandas DataFrames](https://pandas.pydata.org/docs/).

Available results for the network are:

* `res_buses`: Buses potentials indexed by *(bus id, phase)*
* `res_buses_voltages`: Buses voltages indexed by *(bus id, voltage phase)*
* `res_branches`: Branches currents, powers, and potentials indexed by *(branch id, phase)*
* `res_lines_losses`: Lines series, shunt, and total losses indexed by *(line id, phase)*
* `res_loads`: Loads currents, powers, and potentials indexed by *(load id, phase)*
* `res_loads_voltages`: Loads voltages indexed by *(load id, voltage phase)*
* `res_loads_flexible_powers`: Loads flexible powers (only for flexible loads) indexed by
  (load id, phase)
* `res_sources`: Sources currents, powers, and potentials indexed by *(source id, phase)*
* `res_grounds`: Grounds potentials indexed by *ground id*
* `res_potential_refs`: Potential references currents indexed by *potential ref id* (always zero
  for a successful load flow)

All the results are complex numbers. You can always access the magnitude of the results using
the `abs` function and the angle in radians using the `np.angle` function. For instance,
`abs(network.res_loads)` gives you the magnitude of the loads' results in SI units.

Below are the results of the load flow for `en`:

In [ ]:
en.res_buses

In [ ]:
en.res_buses_voltages

In [ ]:
en.res_branches

In [ ]:
en.res_lines_losses

In [ ]:
en.res_loads

In [ ]:
en.res_loads_voltages

In [ ]:
en.res_sources

In [ ]:
en.res_grounds

In [ ]:
en.res_potential_refs

## Updating elements of the network

Network elements can then be updated. Here, the load constant power values are changed. We
create an unbalanced situation.

In [ ]:
# Change the load to an unbalanced one then rerun the load flow
load.powers = [15e3, 0, 0]  # VA
en.solve_load_flow(auth=auth)
load_bus.res_potentials

One can notice that the neutral's potential of the bus is no longer close to 0 V.

## Saving the network to a file

The network can be saved to a JSON file using the `en.to_json` method. Note that this method
does not save the results of the load flow. It only saves the network elements.

To save the results of the load flow, use the `en.results_to_json` method.

```python
en.to_json("my_network.json")
en.results_to_json("my_network_results.json")
```

<div class="alert alert-danger">

**Warning:**

The `to_json` and `results_to_json` methods will overwrite the file if it already exists.
</div>

## Loading a network from a file

A saved network can be loaded using the `ElectricalNetwork.from_json` method. The results of
the load flow can then be loaded using the `ElectricalNetwork.results_from_json` method.

```python
en = ElectricalNetwork.from_json("my_network.json")
en.results_from_json("my_network_results.json")
```